## QA-Loader

In [1]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

os.chdir('..')

In [2]:
from textlong import LocalFilesLoader, QAExcelsLoader, QAMarkdownLoader

In [3]:
qa = QAExcelsLoader()
pages = qa.load()
print(len(pages))

50


In [4]:
print(pages[0].page_content)
print()
print(pages[0].metadata)
print(pages[0].metadata['answer'])

低层建筑 
场景：大于300平方米的银行营业厅
问题：是否需要设置自动喷水系统？

{'answer': '问题答案：需要设立自动喷水灭火系统。\n\n相关规范解释：《建筑设计防火规范》（GB 50016-2014）第5.3.1条规定，公共建筑中的人员密集场所应设置自动喷水灭火系统。银行营业厅作为公共场所，且面积大于300平米，符合此规范要求。\n\n', 'source': '/Users/xuehongwei/github/textlong/notes/data/public/__QA__/消防50条问题(240520).xlsx', 'sheet': 'QA_50个问题'}
问题答案：需要设立自动喷水灭火系统。

相关规范解释：《建筑设计防火规范》（GB 50016-2014）第5.3.1条规定，公共建筑中的人员密集场所应设置自动喷水灭火系统。银行营业厅作为公共场所，且面积大于300平米，符合此规范要求。




In [3]:
from langchain_core.documents import Document
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

docs = QAExcelsLoader().load()

embeddings = OpenAIEmbeddings()
db = FAISS.from_documents(docs, embeddings)
retriever = db.as_retriever()

In [6]:
from textlong import format_qa_docs
retriever.invoke("商场扶梯下面要设置喷水吗？")

[Document(page_content='商场\n场景：商场店铺内钢楼梯下方（非扶梯）\n问题：需要设置喷头吗？', metadata={'answer': '问题答案：需要设置喷头。\n\n相关规范解释：根据《建筑设计防火规范》（GB 50016-2014）和《自动喷水灭火系统设计规范》（GB 50084-2001），商场店铺内的钢楼梯下方应设置自动喷水灭火系统的喷头，以确保在火灾发生时能够迅速有效地控制火势。钢楼梯下方可能存在电线、管道等易燃物品，且火灾蔓延速度快，因此需要喷头覆盖以增强防火能力。', 'source': '/Users/xuehongwei/github/textlong/notes/data/public/__QA__/消防50条问题(240520).xlsx', 'sheet': 'QA_50个问题'}),
 Document(page_content='商场\n场景：商场自动扶梯下方\n问题：需要设置自动喷淋灭火系统吗？', metadata={'answer': '问题答案：通常不需要。\n\n相关规范解释：根据《建筑设计防火规范》（GB 50016-2014）和《自动喷水灭火系统设计规范》（GB 50084-2001），自动扶梯下方并不被视为通常需要设置自动喷淋系统的区域。自动扶梯下方的空间一般较小，且不是主要的火灾荷载区域。然而，如果自动扶梯附近有其他火灾风险较高的区域或设施，如电气室、机械设备等，则应按照规范的相关要求考虑设置自动喷淋系统。', 'source': '/Users/xuehongwei/github/textlong/notes/data/public/__QA__/消防50条问题(240520).xlsx', 'sheet': 'QA_50个问题'}),
 Document(page_content='低层建筑 \n场景：大于300平方米的银行营业厅\n问题：是否需要设置自动喷水系统？', metadata={'answer': '问题答案：需要设立自动喷水灭火系统。\n\n相关规范解释：《建筑设计防火规范》（GB 50016-2014）第5.3.1条规定，公共建筑中的人员密集场所应设置自动喷水灭火系统。银行营业厅作为公共场所，且面积大于300平米，符合此规范要求。\n\n', 'source': '/Us

In [15]:
from langchain_zhipu import ChatZhipuAI
from textlong import format_qa_docs
from textlong.prompts import create_qa_prompt, load_chat_prompt
from textlong.memory import MemoryManager, WithMemoryBinding

# prompt = create_qa_prompt()
prompt = load_chat_prompt("qa", "xiaofang")
llm = ChatZhipuAI()
retriever = db.as_retriever()

chain = {
    "context":  (lambda x: x['input']) | retriever | format_qa_docs,
    "question": lambda x: x['input'],
    "history":  lambda x: x['history'],
} | prompt | llm

memory = MemoryManager()
withMemoryChain = WithMemoryBinding(chain, memory)
config = {"configurable": {"session_id": "1"}}

In [16]:
# chain = create_qa_chain(llm, retriever)
for x in withMemoryChain.stream({"input": "商场扶梯下面要设置喷水吗？"}, config):
    print(x.content, end="|")

不需要|。

相关|规范|解释|：|根据|提供的|资料|，|没有|明确|规范|要求|商场|扶|梯|下方|必须|设置|喷|水|系统|。|实际上|，|自动|扶|梯|下方|通常|不是|主要的|火灾|荷载|区域|，|因此|不需要|特别|设置|喷|水|系统|。|然而|，|如果|扶|梯|附近|有|其他|火灾|风险|较高的|区域|或|设施|，|应|按|规范|要求|考虑|是否|设置|喷|水|系统|。||input_messages: [HumanMessage(content='商场扶梯下面要设置喷水吗？')]
store: Human: 商场扶梯下面要设置喷水吗？
AI: 不需要。

相关规范解释：根据提供的资料，没有明确规范要求商场扶梯下方必须设置喷水系统。实际上，自动扶梯下方通常不是主要的火灾荷载区域，因此不需要特别设置喷水系统。然而，如果扶梯附近有其他火灾风险较高的区域或设施，应按规范要求考虑是否设置喷水系统。


In [17]:
# chain = create_qa_chain(llm, retriever)
for x in withMemoryChain.stream({"input": "下面有储物间呢？"}, config):
    print(x.content, end="|")

如果|商场|扶|梯|下面|有|储|物|间|，|那么|需要|根据|储|物|间|的大小|、|用途|及其|它|相关|因素|来|决定|是否|设置|喷|水|系统|。

相关|规范|解释|：|根据|《|建筑设计|防火|规范|》（|GB| |500|16|-|201|4|）|和|《|自动|喷|水|灭火|系统|设计|规范|》（|GB| |500|84|-|200|1|），|储|物|间的|防火|设计|要求|会|考虑到|其|火灾|荷载|、|面积|、|存储|物品|的类型|等因素|。|如果|储|物|间|面积|较大|或|存储|的|物品|易|燃|，|可能|需要|设置|自动|喷|水|灭火|系统|。

具体|是否|需要|设置|，|需要|依据|以下|条件|评估|：
-| 储|物|间的|面积|；
-| 储|物|间|内|物品|的|火灾|危险性|；
-| 储|物|间|与|扶|梯|之间的|防火|隔离|情况|；
-| 建|筑|的整体|消防|设计|要求|。

建议|在实际|操作|中|咨询|专业|消防|设计师|或|相关部门|，|以确保|符合|当地|消防|法规|和|标准|。||input_messages: [HumanMessage(content='下面有储物间呢？')]
store: Human: 商场扶梯下面要设置喷水吗？
AI: 不需要。

相关规范解释：根据提供的资料，没有明确规范要求商场扶梯下方必须设置喷水系统。实际上，自动扶梯下方通常不是主要的火灾荷载区域，因此不需要特别设置喷水系统。然而，如果扶梯附近有其他火灾风险较高的区域或设施，应按规范要求考虑是否设置喷水系统。
Human: 下面有储物间呢？
AI: 如果商场扶梯下面有储物间，那么需要根据储物间的大小、用途及其它相关因素来决定是否设置喷水系统。

相关规范解释：根据《建筑设计防火规范》（GB 50016-2014）和《自动喷水灭火系统设计规范》（GB 50084-2001），储物间的防火设计要求会考虑到其火灾荷载、面积、存储物品的类型等因素。如果储物间面积较大或存储的物品易燃，可能需要设置自动喷水灭火系统。

具体是否需要设置，需要依据以下条件评估：
- 储物间的面积；
- 储物间内物品的火灾危险性；
- 储物间与扶梯之间的防火隔离情况；
- 建筑的整体消防设计要求。

建议在实际操作中咨询专业消防设计师或相关部门，以确保符合当地消防法规和标

In [13]:
create_qa_prompt()

ChatPromptTemplate(input_variables=['context', 'history', 'question'], input_types={'history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'task_instruction': '\n你是一名咨询专家，只负责根据资料回答相关提问，禁止回答与此无关的问题。\n\n1. 如果你获得的参考例子无法回答问题，可以查询互联网，但务必注意资料的真实性，不要做任何编造\n2. 请使用简洁的语言回答，不要啰嗦\n3. 不要生成"根据提供的资料..."等字眼\n', 'output_format': '\n输出样例：\n```\n问题答案：xxx。\n\n相关规范解释：xxxxxxxx\n```\n'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['task_instruction'], template='{{task_instruction}}', template_format='mustache')), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='我有哪些资料可以参考？', template_format='mustache')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'output_f

In [12]:
load_chat_prompt("qa", "xiaofang")

ChatPromptTemplate(input_variables=['history'], input_types={'history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'output_format': '\n输出样例：\n```\n问题答案：xxx。\n\n相关规范解释：xxxxxxxx\n```', 'task_instruction': '\n你是一名消防咨询专家，只负责回答消防领域的相关提问，禁止回答无关问题。\n\n1. 如果你获得的参考例子无法回答问题，可以查询互联网，但务必注意资料的真实性，不要做任何编造\n2. 请使用简洁的语言回答，不要啰嗦\n3. 不要生成”根据提供的资料...“等字眼\n'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='{{task_instruction}}')), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='我有哪些资料可以参考？')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='你可以参考这些资料：\n{{context}}\n你必须按照如下格式输出：{{output_format}}')), AIMessagePromptTemplate(prompt=PromptTemplate(i

In [13]:
from textlong.prompts import save_chat_prompt, load_chat_prompt
save_chat_prompt(prompt, "qa", "xiaofang")

In [6]:
query = "有什么例子"
results = db.similarity_search(query)
print(results)
# for doc, score in results:
#     print(f"Content: {doc.page_content}, Metadata: {doc.metadata}, Score: {score}") 

[Document(page_content='Sample text 1', metadata={'source': 'file1.txt'}), Document(page_content='Sample text 2', metadata={'source': 'file2.txt'})]


In [6]:
%pip install faiss-cpu

  Using cached faiss_cpu-1.8.0-cp310-cp310-macosx_10_14_x86_64.whl.metadata (3.6 kB)
Using cached faiss_cpu-1.8.0-cp310-cp310-macosx_10_14_x86_64.whl (7.3 MB)
Note: you may need to restart the kernel to use updated packages.


In [5]:
from textlong.prompts import save_chat_prompt, load_chat_prompt, create_writing_todo_prompt
save_chat_prompt(create_writing_todo_prompt(), "todo", "xiaofang")

In [6]:
load_chat_prompt("todo", "xiaofang")

ChatPromptTemplate(input_variables=['history', 'howto', 'outline_exist', 'task', 'title', 'words_advice'], input_types={'history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'json_instruction': '1. 你只能输出一个JSON段落，否则我将无法正确解析。2. 你必须严格遵循我提出的JSON键值规则，不要额外发挥，否则我将无法正确解析。3. 如果你的创作中出现实体名称、创作设定等，就将其单独提炼到扩写指南或内容摘要；这样做非常必要，可以让独立的创作子任务保持一致的背景设定。', 'output_format': '\n（请确保输出符合JSON语法限定，并且不要出现其他的JSON键，以便我能够正确解析）\n```json\n{\n    "详细内容": [str类型]你的详细输出,\n    "内容摘要": [str类型]详细内容提要，可以包括涉及的人物、地点、情节等实体名称和背景设定\n}\n```\n', 'task_instruction': '你是一名优秀的写手，负责详细构思段落细节。'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['json_instruction', 'output_format', 'task_instruction'], template='\n{{task_instruction}}\n请务必记住：{{json_i